In [1]:
import urllib
import os
import zipfile

import numpy as np

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Conjunto de Dados

In [2]:
reviews = np.loadtxt("data/analise-sent/yelp_reviews_no_score.txt", delimiter='\n', dtype=str)
sentiment = np.loadtxt("data/analise-sent/yelp_reviews_so_score.txt", dtype=int)

## Análise de Sentimentos

Nós estamos interessados em predizer o sentimento dos tweet a partir de seu contéudo. Portanto, devemos construir um modelo que seja capaz de classificar o sentimento (empty, sadness, enthisuasm, neutral) de um tweet a partir de seu texto.

### Desafio

Você deve a paritr dos conhecimentos adquiridos nas aula teórica, e seguindo os exemplos de classificação de texto exibidos em alguns notebooks: 
- machine_learning/aula_01/svm.ipynb
- machine_learning/aula_02/naive_bayes.ipynb
- machine_learning/aula_03/selecao_validacao.ipynb

Construir um classificador capaz de predizer o sentimento de um dado tweet. Você deve utilizar tentar utilzar todo pipeline do scikit-learn para transformar os dados, selecionar e treinar o modelo.

In [3]:
# TODO: Dividir conjunto de dados em treino e teste

# TODO: Construir pipeline para execução como em machine_learning/aula_03/selecao_validacao.ipynb 

# TODO: Testar vários alogirtmo para classificação do sentimento (análise de sentimentos) de tweets
# Ex: LinearSVC, Naïve Bayes, RandomForest, SGB, Regressão Logística, Perceptron, ensembles...
# Utilizar as técnicas de seleção de modelo vista em machine_learning/aula_03/selecao_validacao.ipynb

### Enriquecendo os dados com NLTK e SentiWordNet

[NLTK](http://www.nltk.org/) é uma plataforma criar programas em Python para trabalhar com dados de linguagem humana. Ela provê interfaces fáceis para mais de 50 corpus textuais e recursos léxicos tal como WordNet, juntamente com bibliotecas para processamente textual para classificação, tokenização, stemming, tagging, parsing, e semantica.

WordNet nos prove uma interface, através do NLTK, para fazermos análise léxicas (descubrir se um termo é adjetivo, substantivo entre outros). O WordNet possui um dicionário de sentimentos, o SentiWordNet. Com isso conseguimos dar mais informações para o algoritmo de aprendizado e assim, podemos (talvez) aumentar seu poder preditivo.

Vamos ver um exemplo de uso desses recursos.

In [4]:
# carregando a biblioteca do SentiWordNet através do nltk
from nltk.corpus import sentiwordnet as swn

happy = swn.senti_synsets('happy', 'a')
list(happy)

[SentiSynset('happy.a.01'),
 SentiSynset('felicitous.s.02'),
 SentiSynset('glad.s.02'),
 SentiSynset('happy.s.04')]

Vamos pegar a primeira ocorrência ```SentiSynset('happy.a.01')``` e destiricá-la:

- happy = palavra que nós precisamos do score
- a = classe gramatical (i.e., adjetivo, substantivo...)
- '01' = Uso (01 para o uso mais comum e um número mais alto indica baixo uso da palavra)

Com isso em mente, podemos ver que ```swn.senti_synsets('happy', 'a')``` busca por formas de usado da palavra 'happy' como adjetivo, e retorna o resultado ordenado pelo uso mais comum.

As classes gramaticais suportadas são:

- n - NOUN (substantivo)
- v - VERB (verbo)
- a - ADJECTIVE (adjetivo)
- s - ADJECTIVE SATELLITE (?) 
- r - ADVERB (adverbio)

In [5]:
word = 'happy'
# score negativo para o advérbio happily
print("pos_score(%s): " % word, list(swn.senti_synsets(word, 'a'))[0].pos_score())
# score negativo para o advérbio happily
print("neg_score(%s): " % word, list(swn.senti_synsets(word, 'a'))[0].neg_score())

word = 'happily'
# score negativo para o advérbio happily
print("pos_score(%s): " % word, list(swn.senti_synsets(word, 'r'))[0].pos_score())
# score negativo para o advérbio happily
print("neg_score(%s): " % word, list(swn.senti_synsets(word, 'r'))[0].neg_score())

pos_score(happy):  0.875
neg_score(happy):  0.0
pos_score(happily):  0.5
neg_score(happily):  0.25


Caso não a palavra não seja da classe gramativa em questão ou então não exista o método retorna uma lista vazia.

In [6]:
# happy é adjetivo e não um advérbio
list(swn.senti_synsets('happy', 'r'))

[]

A ideia então é enriquecer seu modelo com essas infomações. Por exemplo, ao invés de utilizar frequência de termo, você pode utilizar quão positivo é aquele termo e quão negativo.

In [7]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse

class CountSentimentTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.cv = CountVectorizer(stop_words = 'english')
        
    def fit(self, X, y):
        
        def sentiment(term):
            """Para cada termo é recuperado quão positivo ou negativo ele é
            de acordo com o SentiWordNet"""
            part_of_speech = ['a', 'r', 'v']
            pos = neg = obj = 0
            
            try:
                for p in part_of_speech:
                    results = list(swn.senti_synsets(term, p))
                    if results != []:
                        pos += results[0].pos_score()
                        neg += results[0].neg_score()
                        obj += results[0].obj_score()
            except:
                pass
                
            # é retornado um array com os scores para cada sentimento            
            return np.array([pos, neg, obj])
    
        
        # fitamos o CountVectorizer para extração dos termos
        self.cv.fit(X, y)
        
        # para cada termo, nós obtemos a intensidade de seus sentimentos
        # utilizando o método sentiment acima.
        indices = [int(index) for term, index in self.cv.vocabulary_.items()]
        terms = np.array([term for term, index in self.cv.vocabulary_.items()])
        self.sent = np.array([sentiment(term) for term, index in self.cv.vocabulary_.items()])
        
        return self
    
    def transform(self, X):
        # transformando os dados de acordo com
        # CountVectorizer aprendido
        X_t = self.cv.transform(X)
        
        # Multiplicamos a ocorrencia dos termos pela intensidade
        # de cada sentimento (positivo e negativo)
        X_pos = X_t.multiply(self.sent[:, 0])
        X_neg = X_t.multiply(self.sent[:, 1])
        X_obj = X_t.multiply(self.sent[:, 2])
        
        # Concatenamos os atributos enriquecidos e os atributos originais
        return scipy.sparse.hstack((X_pos, X_neg, X_t)).tocsr()


In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import Normalizer


X_train, X_test, y_train, y_test = \
    train_test_split(reviews, sentiment, test_size = 0.4, random_state = 0)


csent = CountSentimentTransformer()
# Criamos um pipeline com nosso CountSentiment, Normalizer e um LinearSVC
# Quando damos um fit no pipeline ele vai passando o resultados dos transformers em cadeia.
# Ou seja, primeiramente é executado o fit_transform do CounSentimentTransformer, os dados
# transformados por esse transformer é passado para o seguinte transformer no pipeline,
# o Normalizer, esse por sua vez transforma os vetores em vetores unitários e passa
# para o LinearSVC os dados transformados que por sua vez é treinado.
# Quando o predict é utilizado esse processo de transformação também é executado para o
# dado sendo testado.
pipe = make_pipeline(csent, Normalizer(), LinearSVC())
cv = GridSearchCV(pipe, {'linearsvc__C': 2. ** np.arange(-3, 5, 1)},  verbose=1, n_jobs=-1)

cv.fit(X_train, y_train)
print(cv.best_score_, cv.best_params_)
print(f1_score(cv.predict(X_test), y_test, average = 'macro'))


pipe = make_pipeline(CountVectorizer(stop_words = 'english'), Normalizer(), LinearSVC())
cv = GridSearchCV(pipe, {'linearsvc__C': 2. ** np.arange(-3, 5, 1)}, verbose=1, n_jobs=-1)
cv.fit(X_train, y_train)
print(cv.best_score_, cv.best_params_)
print(f1_score(cv.predict(X_test), y_test, average = 'macro'))

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    9.7s finished


0.913333333333 {'linearsvc__C': 1.0}
0.922496725487
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    3.3s finished


0.912333333333 {'linearsvc__C': 1.0}
0.921998049951
